In [ ]:
import re 
import pandas

In [3]:
DATA_PATH: str = "../../data/processed"

In [65]:
(
    pandas.read_csv(f"{DATA_PATH}/twitter.german.dataset.enriched.csv")
    [["author_id_post", "author_first_name_post", "author_last_name_post", "author_party_post"]]
    .drop_duplicates()
    .set_index("author_id_post")
    .join(
        (
            pandas.read_csv(f"{DATA_PATH}/twitter.german.dataset.enriched.csv")
            .groupby("author_id_post")
            ["topics_post"]
            .aggregate(lambda x: [topic.strip() for topic in set(','.join(x).split(","))])
        )
    )
    .join(
        (
            pandas.read_csv(f"{DATA_PATH}/twitter.german.dataset.enriched.csv")
            ["author_id_post"]
            .value_counts(normalize=True)
            .rename("content_proportion")
        )
    )
    .rename(columns=lambda x: re.sub('_post|author_','',x))
    .reset_index(drop=True)
    .to_json("agents.post.json", orient="records", indent=4, force_ascii=False)
)

In [60]:
(
    pandas.read_csv(f"{DATA_PATH}/twitter.german.dataset.enriched.csv")
    .pipe(lambda df: df.assign(history_item=df["text_post"].str.cat(df["text_reply"], sep="|").str.split("|")))
    .groupby("author_id_reply")
    ["history_item"]
    .aggregate(lambda x: x)
    .to_frame()
    .join(
        (
            pandas.read_csv(f"{DATA_PATH}/twitter.german.dataset.enriched.csv")
            ["author_id_reply"]
            .value_counts(normalize=True)
            .rename("content_proportion")
        )
    )
    .reset_index(drop=True)
    .to_json("agents.reply.json", orient="records", indent=4, force_ascii=False)
)